# Import libraries

In [47]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
import numpy as np
import pickle

# load dataset

load the clean dataset

In [48]:
data = './out_clean2.csv'
df = pd.read_csv(data, encoding = "ISO-8859-1")
df = df.drop("Unnamed: 0",axis=1)

In [69]:
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,saat semua cowok usaha lacak perhati gue kamu ...,1,1,1,0,0,0,0,0,1,1,0,0
1,siapa telat beri tau eluedan sarap gue gaul ci...,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang aku pikir aku tetap percaya tuhan padah...,0,0,0,0,0,0,0,0,0,0,0,0
3,aku akunnku tau mata sipit lihat mana aku,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum cebong kafir lihat dongok awal tambah dun...,1,1,0,1,1,0,0,0,0,0,1,0


In [21]:
y_cols = list(df.columns[1:])

In [22]:
# Inspect the class balance
def get_class_weight(data):
    class_weight = {}
    for num,col in enumerate(y_cols):
        if num not in class_weight:
            class_weight[col] = round((data[data[col] == 1][col].sum())/data.shape[0]*100,2)
    return class_weight
class_weight = get_class_weight(df)
print('Total class weight: ', sum(class_weight.values()), '%\n\n', class_weight)

Total class weight:  208.46999999999997 %

 {'HS': 42.23, 'Abusive': 38.29, 'HS_Individual': 27.15, 'HS_Group': 15.08, 'HS_Religion': 6.02, 'HS_Race': 4.3, 'HS_Physical': 2.45, 'HS_Gender': 2.32, 'HS_Other': 28.4, 'HS_Weak': 25.69, 'HS_Moderate': 12.95, 'HS_Strong': 3.59}


# Split

In [23]:
def get_train_test_val(data):
    X_data = data['Tweet']
    y_data = data[list(data.columns[1:])].values
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, train_size=0.8)

    print(
        'X_train shape', X_train.shape,
        '\ny_train shape', y_train.shape,
        '\nX_test shape', X_test.shape,
        '\ny_test shape', y_test.shape,

    )
    return X_train, X_test, y_train , y_test

divide the data into 4 dataframe
- HS Abusive
- HS Category Religion etc
- HS Target Individual Group
- HS Level W M S

In [24]:
# HS Abusive
df_HS = df[['Tweet', 'HS','Abusive']]

In [25]:
# HS Category
df_HS_cat = df[['Tweet', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']]

In [26]:
#HS Target
df_HS_tar = df[['Tweet', 'HS_Individual' , 'HS_Group']]

In [27]:
# HS Level
df_HS_lvl = df[['Tweet', 'HS_Weak', 'HS_Moderate' , 'HS_Strong']]

## Training

Based on the paper, the best combination was word unigram feature extraction, random forests classifier and label powerset transformation. I tried to reduplicate the research as attempt to learn about multilabel classification

Replace the data in the cells below to train a different dataframe/

In [28]:
data = df_HS_tar
# df_HS, df_HS_cat, df_HS_tar, df_HS_lvl

In [29]:
X_train, X_test, y_train, y_test = get_train_test_val(data)

X_train shape (10535,) 
y_train shape (10535, 2) 
X_test shape (1317,) 
y_test shape (1317, 2)


initiate feature extractor using tf idf and word n-grams 

In [30]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.9, min_df=5)

In [31]:
def tfidf_features(X_train, X_test):
    """
        X_train, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train.apply(lambda x: np.str_(x)))
    # Transform the train, test set and return the result
    X_test_tfidf = tfidf_vectorizer.transform(X_test.apply(lambda x: np.str_(x)))
    
    return X_train_tfidf, X_test_tfidf, tfidf_vectorizer.vocabulary_

extract the feature in dataset

In [32]:
X_train_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

do the training using label powerset data transformation method and random forest classifier

In [33]:
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier

classifier = LabelPowerset(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

classifier.fit(X_train_tfidf, y_train)

LabelPowerset(classifier=RandomForestClassifier(), require_dense=[False, True])

In [34]:
y_hat=classifier.predict(X_test_tfidf)

see the training results by viewing its F1-score

In [35]:
import sklearn.metrics as metrics

In [36]:
lp_f1=metrics.f1_score(y_test, y_hat, average='micro')
lp_hamm=metrics.hamming_loss(y_test,y_hat)
print('Label Powerset F1-score:',round(lp_f1,3))

Label Powerset F1-score: 0.63


In [37]:
res = tfidf_vectorizer.transform(['bani cebong'])

In [38]:
output = classifier.predict_proba(res)
output2 = classifier.predict(res)

In [39]:
print(output.toarray()[0])

[0.1        0.82666667]


In [40]:
print(output2.toarray())

[[0 1]]


## Save the model and vectorizer

In [41]:
import dill as pickle

Just need to change the name each save

In [42]:
#save model
with open('model_Target.sav', 'wb') as fin:
    pickle.dump(classifier, fin)

pickle.dump(tfidf_vectorizer, open("vec_target.pickle", "wb")) #//Save vectorizer

# Predict

In [1]:
import dill as pickle

In [2]:
text = 'dasar goblok'

## HS Abusive

In [3]:
tfidf_vectorizer = pickle.load(open('vec_abusive.pickle', 'rb'))

In [4]:
res = tfidf_vectorizer.transform([text])

In [5]:
model = pickle.load(open('model_abusive.sav', 'rb'))

In [35]:
def predict_abusive(model, vec):
    output = model.predict_proba(vec)
    hs = output.toarray()[0][0]
    ab = output.toarray()[0][1]
    print('Hate Speech: ', hs)
    print('Abusive: ', ab)

In [ ]:
predict_abusive(model, res)

## HS Category

In [7]:
tfidf_vectorizer = pickle.load(open('vec_Category.pickle', 'rb'))

In [8]:
res = tfidf_vectorizer.transform([text])

In [9]:
model = pickle.load(open('model_Category.sav', 'rb'))

In [34]:
def predict_category(model, vec):
    output = model.predict_proba(vec)
    religion = output.toarray()[0][0]
    race = output.toarray()[0][1]
    physical = output.toarray()[0][2]
    gender = output.toarray()[0][3]
    other = output.toarray()[0][4]
    print('Religion: ', religion)
    print('Race: ', race)
    print('Physical: ', physical)
    print('Gender: ', gender)
    print('Other: ', other)

In [ ]:
predict_category(model, res)

## HS Target

In [11]:
tfidf_vectorizer = pickle.load(open('vec_target.pickle', 'rb'))

In [12]:
res = tfidf_vectorizer.transform([text])

In [13]:
model = pickle.load(open('model_Target.sav', 'rb'))

In [33]:
def predict_target(model, vec):
    output = model.predict_proba(vec)
    ind = output.toarray()[0][0]
    gr = output.toarray()[0][1]
    print('Individual: ', ind)
    print('Group: ', gr)

In [ ]:
predict_target(model, res)

## HS Level

In [15]:
tfidf_vectorizer = pickle.load(open('vec_level.pickle', 'rb'))

In [16]:
res = tfidf_vectorizer.transform([text])

In [17]:
model = pickle.load(open('model_Level.sav', 'rb'))

In [32]:
def predict_level(model, vec):
    output = model.predict_proba(vec)
    weak = output.toarray()[0][0]
    moderate = output.toarray()[0][1]
    strong = output.toarray()[0][2]
    print('Weak: ', weak)
    print('Moderate: ', moderate)
    print('Strong: ', strong)

In [19]:
predict_level(model, res)

Weak:  0.73
Moderate:  0.15
Strong:  0.0


# Combined

In [26]:
modelA = pickle.load(open('model_abusive.sav', 'rb'))
modelC = pickle.load(open('model_Category.sav', 'rb'))
modelT = pickle.load(open('model_Target.sav', 'rb'))
modelL = pickle.load(open('model_Level.sav', 'rb'))

In [28]:
vecA = pickle.load(open('vec_abusive.pickle', 'rb'))
vecC = pickle.load(open('vec_Category.pickle', 'rb'))
vecT = pickle.load(open('vec_Target.pickle', 'rb'))
vecL = pickle.load(open('vec_Level.pickle', 'rb'))

In [41]:
def predictAll(text):
    resA = vecA.transform([text])
    resC = vecC.transform([text])
    resT = vecT.transform([text])
    resL = vecL.transform([text])
    predict_abusive(modelA, resA)
    predict_category(modelC, resC)
    predict_target(modelT, resT)
    predict_level(modelL, resL)

In [67]:
text = 'dasar cina bodoh'

In [68]:
predictAll(text)

Hate Speech:  0.5725
Abusive:  0.6599999999999999
Religion:  0.05
Race:  0.48
Physical:  0.0
Gender:  0.05
Other:  0.19
Individual:  0.7
Group:  0.08
Weak:  0.616
Moderate:  0.1
Strong:  0.02
